# Multi-level Linear Mixed Effects Regression

Demo for the NAG function [`naginterfaces.library.correg.lmm_fit`](https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.correg.html#naginterfaces.library.correg.lmm_fit) to showcase a Multi-level linear mixed effects regression model using restricted maximum likelihood.

This demo illustrates the combined use of

 * `naginterfaces.library.correg.lmm_init`,
 * `naginterfaces.library.correg.lmm_fit`,
 * `naginterfaces.library.blgm.lm_formula`,
 * `naginterfaces.library.blgm.lm_submodel`,
 * `naginterfaces.library.blgm.lm_describe_data`, and
 * `naginterfaces.library.blgm.handle_free`.
 
For full information please refer to the NAG Library document for 
[`naginterfaces.library.correg.lmm_fit`](https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.correg.html#naginterfaces.library.correg.lmm_fit) or [`g02jh`](https://www.nag.com/numeric/nl/nagdoc_27.1/flhtml/g02/g02jhf.html).

The demo fits a random effects model with three random submodels and two fixed effects to a simulated dataset with 90 observations and 12 variables. The model is fit using restricted maximum likelihood (REML) and is of the form

 $$y=Xβ+Zν+ε, $$

where
 * $y$ is a vector of $n$ observations on the dependent variable,
 * $X$ is a known $n×p$ design matrix for the fixed independent variables,
 * $β$ is a vector of length $p$ of unknown fixed effects,
 * $Z$ is a known $n×q$ design matrix for the random independent variables,
 * $ν$ is a vector of length $q$ of unknown random effects, and
 * $ε$ is a vector of length $n$ of unknown random errors.
 
## For more details
 * Model related routines consult [here](https://www.nag.com/numeric/nl/nagdoc_latest/flhtml/g02/g02jhf.html#description).
 * **Modelling language** details and **syntax** see [[Python doc](https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.blgm.lm_formula.html#naginterfaces-library-blgm-lm-formula) | [Full doc](https://www.nag.com/numeric/nl/nagdoc_latest/flhtml/g22/g22yaf.html#description)]
 * See [Chapter G02, Section 2.6](https://www.nag.com/numeric/nl/nagdoc_latest/flhtml/g02/g02intro.html#background210) to read more about linear mixed effects regression.
 * See [Chapter G22](https://www.nag.com/numeric/nl/nagdoc_latest/flhtml/g22/g22intro.html) for help on the modeling-related functions.



In [1]:
from collections import namedtuple
import warnings
import numpy as np
import pandas as pd
from naginterfaces.base import utils
from naginterfaces.library import blgm, correg

In [2]:
def dataset_meta_data():
    """
    Return some meta data for the dataset
    """
    # The levels and variable names associated with the independent
    # variables:
    levels = [2, 3, 2, 3, 2, 3, 1, 4, 5, 2, 3, 3]
    vnames = ['V' + str(x + 1) for x in range(len(levels))]

    # Number of observations (in this context, this is not really used, so
    # can take any value) we have set it here to the number of observations
    # in the full dataset because we readily know it
    nobs = 90

    return namedtuple(
        'meta_data',
        ['nobs', 'levels', 'vnames'])(
            nobs, levels, vnames
        )

In [3]:
def get_vinfo(what, hlmm):
    """
    Call blgm.lm_submodel to get descriptive information on
    estimates returned from fitting the linear mixed effects
    regression model defined in hlmm
    """

    # supply a dummy submodel handle
    hform = utils.Handle()

    # Find how long vinfo needs to be
    lvinfo = 3
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', utils.NagAlgorithmicWarning)
        subres = blgm.lm_submodel(what, hform, hlmm, 0, 0, lvinfo)

    lvinfo = subres.vinfo[2]

    # Get and return vinfo
    subres = blgm.lm_submodel(what, hform, hlmm, 0, 0, lvinfo)
    return subres.vinfo

In [4]:
def get_next_label(vinfo, vnames=None, fixed_len=None):
    """
    Return a label based on the information supplied in vinfo
    Routine should be called for all estimates described by
    vinfo, in order. vinfo is updated in place in preparation
    for the next call
    """
    k = 0
    b = vinfo[k]
    k += 1

    if b < 0:
        # the corresponding column of the design matrix was padding
        label = ''

    elif b == 0:
        label = 'Intercept'

    else:
        label = ''
        for _i in range(b):
            # Extract the information from VINFO
            # (we ignore the contrast information)
            vi, li, _ = vinfo[k:k+3]
            k += 3

            # Variable name
            if vnames is None:
                this_vname = 'V' + str(vi)
            else:
                # vi'th variable supplied when describing the dataset
                this_vname = vnames[vi - 1]

            # Construct the label
            if label != '':
                label += ' . '

            label += this_vname
            if li > 0:
                label += ' (Lvl ' + str(li) + ')'

    if fixed_len is not None:
        label = label.ljust(fixed_len)

    # shift vinfo in place, so it is ready to produce the
    # next label
    np.copyto(vinfo, np.roll(vinfo, -k))

    return label

## Linear mixed effects regression model using REML

In [5]:
# Get the dataset
# y is the dependent data while dat1..dat12 is the independent data
df = pd.read_csv('lmm_data.csv')
df.head()

,y,dat1,dat2,dat3,dat4,dat5,dat6,dat7,dat8,dat9,dat10,dat11,dat12
0,3.1100,1.0,3.0,2.0,1.0,2.0,2.0,-0.3160,4.0,2.0,1.0,1.0,1.0
1,2.8226,1.0,1.0,1.0,3.0,1.0,2.0,-1.3377,1.0,4.0,1.0,1.0,1.0
2,7.4543,1.0,3.0,1.0,3.0,1.0,3.0,-0.7610,4.0,2.0,1.0,1.0,1.0
3,4.4313,2.0,3.0,2.0,1.0,1.0,3.0,-2.2976,4.0,2.0,1.0,1.0,1.0
4,6.1543,2.0,2.0,1.0,3.0,2.0,3.0,-0.4263,2.0,1.0,1.0,1.0,1.0


In [6]:
# Get the meta data for the dataset
meta = dataset_meta_data()
hddesc = utils.Handle()
blgm.lm_describe_data(hddesc, meta.nobs, meta.levels, vnames=meta.vnames)

For more information on the **modeling language syntax** see [here](https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.blgm.lm_formula.html#naginterfaces-library-blgm-lm-formula).

In [7]:
# Specify the fixed part of the model:
hfixed = utils.Handle()
blgm.lm_formula(hfixed, 'V1 + V2')

# Specify the random part of the model:
hrndm = [utils.Handle() for i in range(3)]
blgm.lm_formula(hrndm[0], '-1 + V7 + V8 + V9 / SUBJECT = V12')
blgm.lm_formula(hrndm[1], '-1 + V5 + V6 / SUBJECT = V12.V11')
blgm.lm_formula(hrndm[2], '-1 + V3 + V4 / SUBJECT = V12.V11.V10')

In [8]:
# Handle to hold a description of the mixed effects model
hlmm = utils.Handle()

In [9]:
# In this contrived demo we will be processing the data in various sized blocks
blk_size = [10, 31, 40, 9]

In [10]:
# Process all the blocks of data
fobs = 0
for nobs in blk_size:
    lobs = fobs + nobs

    # Process the current block of data
    # Make sure **data is contiguous**
    y = df['y'][fobs:lobs]
    obs = df.loc[fobs:lobs,'dat1':].copy()
    pdat = correg.lmm_init(hlmm, hddesc, hfixed, y, obs, hrndm=hrndm)

    if fobs == 0:
        xcomm = pdat.comm.copy()
    else:
        correg.lmm_init_combine(hlmm, xcomm, pdat.comm)

    fobs = lobs

In [11]:
# Initial values for the variance components
# the number of variance components (nvpr) is not data
# dependent, so we can get it from pdat for any of the data blocks
gamma = [-1.0] * (pdat.nvpr + 1)

In [12]:
# Fit the model
fit = correg.lmm_fit(hlmm, gamma, xcomm)

In [13]:
# Get descriptive information on the estimates for the
# fixed parameters, random parameters and variance components
fixed_vinfo = get_vinfo('X', hlmm)
random_vinfo = get_vinfo('Z', hlmm)
vc_vinfo = get_vinfo('V', hlmm)

# The number of random and fixed factors for the full dataset
# (the ones returned by lmm_init are for a particular subset of data):
nff = correg.optget('NFF', xcomm)['value']
nrf = correg.optget('NRF', xcomm)['value']
rnlsv = correg.optget('RNLSV', xcomm)['value']
fnlsv = correg.optget('FNLSV', xcomm)['value']
nxx = nff*fnlsv
nzz = nrf*rnlsv

In [14]:
# Display the results
pad = '   '
print('\nRandom Parameter Estimates')
print('==========================')
print(' Estimate Standard    Label')
print('           Error')
for p in range(nzz):
    label = get_next_label(random_vinfo, meta.vnames)
    if label != '':
        print('{:8.3f}'.format(fit.b[p]) + '{:8.3f}'.format(fit.se[p]) +
              pad + label)

print('\nFixed Parameter Estimates')
print('=========================')
print(' Estimate Standard    Label')
print('           Error')
for p in range(nxx):
    label = get_next_label(fixed_vinfo, meta.vnames)
    if label != '':
        print('{:8.3f}'.format(fit.b[p+nzz]) +
              '{:8.3f}'.format(fit.se[p+nzz]) +
              pad + label)

print('\nVariance Components')
print('===================')
print('  Estimate      Label')
for p in range(pdat.nvpr):
    print('{:9.3f}'.format(fit.gamma[p]) + pad +
          get_next_label(vc_vinfo, meta.vnames))

print('\nSigma^2           = {:15.3f}'.format(fit.gamma[pdat.nvpr]))
print('-2 Log Likelihood = {:15.3f}'.format(fit.lnlike))


Random Parameter Estimates
 Estimate Standard    Label
           Error
   0.683   0.506   V7 . V12 (Lvl 1)
   1.596   1.321   V8 (Lvl 1) . V12 (Lvl 1)
  -0.753   1.566   V8 (Lvl 2) . V12 (Lvl 1)
   0.403   1.684   V8 (Lvl 3) . V12 (Lvl 1)
  -0.852   1.752   V8 (Lvl 4) . V12 (Lvl 1)
   0.570   1.624   V9 (Lvl 1) . V12 (Lvl 1)
   0.001   1.911   V9 (Lvl 2) . V12 (Lvl 1)
  -0.285   1.925   V9 (Lvl 3) . V12 (Lvl 1)
   0.447   2.033   V9 (Lvl 4) . V12 (Lvl 1)
   0.003   2.139   V9 (Lvl 5) . V12 (Lvl 1)
  -0.488   2.821   V5 (Lvl 1) . V11 (Lvl 1) . V12 (Lvl 1)
   1.390   2.936   V5 (Lvl 1) . V11 (Lvl 2) . V12 (Lvl 1)
   1.735   3.137   V5 (Lvl 1) . V11 (Lvl 3) . V12 (Lvl 1)
   1.883   2.753   V5 (Lvl 2) . V11 (Lvl 1) . V12 (Lvl 1)
  -1.573   2.891   V5 (Lvl 2) . V11 (Lvl 2) . V12 (Lvl 1)
  -1.617   3.171   V5 (Lvl 2) . V11 (Lvl 3) . V12 (Lvl 1)
   0.925   3.775   V6 (Lvl 1) . V11 (Lvl 1) . V12 (Lvl 1)
   0.211   3.997   V6 (Lvl 1) . V11 (Lvl 2) . V12 (Lvl 1)
  -1.110   3.937   V6 (Lvl 1) .

In [15]:
# Free the rest of the blgm handles:
map(blgm.handle_free, [hfixed, hlmm] + hrndm)